In this notebook we write some tests while implementing the new deepmod version.

In [41]:
import numpy as np
import pandas as pd
import torch

from DeePyMoD_SBL.data import Burgers
from DeePyMoD_SBL.deepymod_torch.library_functions import library_1D_in
from DeePyMoD_SBL.deepymod_torch.DeepMod import DeepMod, DeepModDynamic
from DeePyMoD_SBL.deepymod_torch.output import Tensorboard, progress
from DeePyMoD_SBL.deepymod_torch.losses import reg_loss, mse_loss
from DeePyMoD_SBL.deepymod_torch.training import train, train_deepmod
from DeePyMoD_SBL.Lasso import lasso
from DeePyMoD_SBL.deepymod_torch.sparsity import scaling, threshold

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

import matplotlib.pyplot as plt


import time


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Making data

In [2]:
x = np.linspace(-2, 5, 75)
t = np.linspace(0.5, 5.0, 25)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

In [3]:
dataset = Burgers(0.1, 1.0)

In [4]:
u = dataset.solution(x_grid, t_grid)

In [5]:
X = np.concatenate((t_grid.reshape(-1, 1), x_grid.reshape(-1, 1)), axis=1)
y = u.reshape(-1, 1)

In [6]:
X_train = torch.tensor(X, dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y, dtype=torch.float32, requires_grad=True)

In [7]:
noise_level = 0.05

In [8]:
X_train = torch.tensor(X, dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y + noise_level * np.std(y) * np.random.randn(y.size, 1), dtype=torch.float32, requires_grad=True)

# Running MSE and PI only with new method

In [9]:
config = {'n_in': 2, 'hidden_dims': [30, 30, 30, 30, 30], 'n_out': 1, 'library_function':library_1D_in, 'library_args':{'poly_order':2, 'diff_order': 2}, 'fit_method': 'lstsq'}

In [10]:
model = DeepModDynamic(**config)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True)

In [13]:
%%time
data = X_train
target = y_train
max_iterations = 5000
loss_func_args={'l1':torch.tensor(1e-5), 'conv_tol': torch.tensor(1e-3)}


print('| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |')
for iteration in torch.arange(0, max_iterations + 1):
    # Calculating prediction and library and scaling
    prediction, time_deriv_list, sparse_theta_list, coeff_vector_list, theta = model(data)
    coeff_vector_scaled_list = scaling(coeff_vector_list, sparse_theta_list, time_deriv_list)
        
    # Calculating loss
    loss_reg = reg_loss(time_deriv_list, sparse_theta_list, coeff_vector_list)
    loss_mse = mse_loss(prediction, target)
    loss = torch.sum(loss_reg) + torch.sum(loss_mse)
        
    # Writing
    if iteration % 100 == 0:
        progress(iteration, 0.0, max_iterations, loss.item(), torch.sum(loss_mse).item(), torch.sum(loss_reg).item(), 0.0)
           
    # Optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
       5000    100.00%               0s   1.10e-04   1.01e-04   9.22e-06   0.00e+00 CPU times: user 1min 14s, sys: 5.72 s, total: 1min 19s
Wall time: 1min 21s


In [14]:
model.fit.coeff_vector

[tensor([[-4.2512e-04],
         [-3.5638e-02],
         [ 9.6511e-02],
         [ 6.9489e-02],
         [-8.6159e-01],
         [-4.4409e-02],
         [-2.3243e-01],
         [ 1.0237e-02],
         [ 9.2692e-03]], grad_fn=<MmBackward>)]

# Running MSE and PI only with old method

In [44]:
config = {'n_in': 2, 'hidden_dims': [30, 30, 30, 30, 30], 'n_out': 1, 'library_function':library_1D_in, 'library_args':{'poly_order':2, 'diff_order': 2}, 'fit_method': 'optim'}

In [45]:
model = DeepModDynamic(**config)

In [36]:
optimizer = torch.optim.Adam(model.network_parameters(), betas=(0.99, 0.999), amsgrad=True)

In [53]:
%%time
data = X_train
target = y_train
max_iterations = 10000
loss_func_args={'l1':torch.tensor(1e-5), 'conv_tol': torch.tensor(1e-3)}


print('| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |')
for iteration in torch.arange(0, max_iterations + 1):
    # Calculating prediction and library and scaling
    prediction, time_deriv_list, sparse_theta_list, coeff_vector_list, theta = model(data)
    coeff_vector_scaled_list = scaling(coeff_vector_list, sparse_theta_list, time_deriv_list)
        
    # Calculating loss
    loss_reg = reg_loss(time_deriv_list, sparse_theta_list, coeff_vector_list)
    loss_mse = mse_loss(prediction, target)
    loss = torch.sum(loss_reg) + torch.sum(loss_mse)
        
    # Writing
    if iteration % 100 == 0:
        progress(iteration, 0.0, max_iterations, loss.item(), torch.sum(loss_mse).item(), torch.sum(loss_reg).item(), 0.0)
           
    # Optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

| Iteration | Progress | Time remaining |     Cost |      MSE |      Reg |       L1 |
       4100     41.00%      1587118208s   5.48e-02   3.68e-02   1.80e-02   0.00e+00 

KeyboardInterrupt: 

In [47]:
model.fit.coeff_vector[0]

Parameter containing:
tensor([[0.0670],
        [0.3093],
        [0.4387],
        [0.6474],
        [0.0215],
        [0.6264],
        [0.8406],
        [0.2168],
        [0.2704]], requires_grad=True)

In [51]:
sparse_coeff_vector_list, sparsity_mask_list = threshold(coeff_vector_list, sparse_theta_list, time_deriv_list)
model.fit.sparsity_mask = sparsity_mask_list
model.fit.coeff_vector = torch.nn.ParameterList(sparse_coeff_vector_list)

In [52]:
model.fit.sparsity_mask

[tensor([ True, False, False,  True, False, False, False, False, False])]